In [1]:
from models.sos import ScalarOnScalarModel
from optimizers.nbdo import NBDO
import numpy as np

In [2]:
nbdo = NBDO(model=ScalarOnScalarModel(Kx=5, criterion = "D", order=1), latent_dim=4, seed=42,)
nbdo.compute_train_set(num_designs=1_000, runs=50)

/Users/thodorisladas/Documents/code/latent_design_labs/src/optimizers/nbdo.py:137: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  X_unit = sampler.random(num_designs).astype(np.float32)


In [3]:
nbdo.fit(epochs=100, patience=10, batch_size=512)
crit, design = nbdo.optimize(n_calls=30)

2025-08-24 04:24:11.208722: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}
2025-08-24 04:24:16.216999: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

In [4]:
crit

14872156721.348185

In [5]:
np.round(design, 2)

array([[-1., -1.,  1.,  1.,  1.],
       [-1.,  1.,  1.,  1., -1.],
       [-1., -1., -1.,  1., -1.],
       [-1., -1., -1., -1.,  1.],
       [-1.,  1.,  1.,  1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [ 1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.],
       [-1., -1.,  1.,  1., -1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.],
       [ 1., -1., -1.,  1.,  1.],
       [-1., -1.,  1., -1.,  1.],
       [ 1.,  1., -1., -1.,  1.],
       [-1., -1., -1.,  1., -1.],
       [-1., -1., -1., -1., -1.],
       [-1.,  1.,  1.,  1.,  1.],
       [-1.,  1., -1.,  1.,  1.],
       [-1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1.,  1.],
       [-1.,  1., -1.,  1., -1.],
       [ 1.,  1., -1.,  1.,  1.],
       [ 1.,  1.,  1., -1., -1.],
       [ 1.,  1.,  1.,  1., -1.],
       [ 1., -1.,  1., -1., -1.],
       [-1.,  1., -1., -1.,  1.],
       [ 1., -1.,  1.,  1.,  1.],
       [-1., -1., -1., -1.,  1.],
       [ 1.,  